In [1]:
#Import the Libraries

In [2]:
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency
import cx_Oracle
import pickle
import db_config
import datetime
import itertools

In [3]:
#Database Connection with python

In [4]:
timestart = datetime.datetime.now()
con = cx_Oracle.connect(db_config.user, db_config.pw, db_config.dsn)
# print("Database version:", con.version)
dfs_query='select * from (select * from (select py_customer_id, customer_id, customer_bank_name, customer_bank_short_name, customer_type, customer_title, customer_name, customer_creation_date, status_code as status_code_mst , customer_status_date, cust_create_from, date_of_birth, customer_age, customer_gender, cust_birth_place, martial_status, education_id, education_desc, const_code, const_desc, occupation_code, occupation_desc, community_code, community_desc, caste, caste_code, nationality_code, nationality_desc, domicile_code, domicile_desc, mobile_no, email_id, aadhaar_card, blood_group, risk_type_id, risk_type_desc, member_flag, pan, tan, form_60_flag, credit_card_flag, debit_card_flag, sms_banking_flag, mobile_banking_flag, net_banking_flag, occupation_details, employed_with, annual_income, guarantee_count, sureity_count, sureity_amount, dmat_accno, nre_nro_nri_flag, permanent_return_date, identification_id, identification_details, issue_date, issued_by, address_proof_id, address_proof_details, documents_given, kyc_complete_flag, kyc_complete_date, kyc_number, establish_licence_flag, establish_licence_no, establish_licence_date, establish_licence_exp, gst_no, gst_reg_date, gst_verify_flag, gst_verify_date, office_mst_id, old_primary_key, permanent_add, official_add, resisdential_add from py_customer_mast) m inner join py_cust_general g on m.py_customer_id=g.py_customer_id inner join py_customer_all_loans l on m.py_customer_id=l.py_customer_id)'#' where PY_CUSTOMER_ID = {}'.format(py_customer_id)
dfs=pd.read_sql_query(dfs_query,con)
df = dfs.loc[:,~dfs.columns.duplicated()]
con.close()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
df

,PY_CUSTOMER_ID,CUSTOMER_ID,CUSTOMER_BANK_NAME,CUSTOMER_BANK_SHORT_NAME,CUSTOMER_TYPE,CUSTOMER_TITLE,CUSTOMER_NAME,CUSTOMER_CREATION_DATE,STATUS_CODE_MST,CUSTOMER_STATUS_DATE,CUST_CREATE_FROM,DATE_OF_BIRTH,CUSTOMER_AGE,CUSTOMER_GENDER,CUST_BIRTH_PLACE,MARTIAL_STATUS,EDUCATION_ID,EDUCATION_DESC,CONST_CODE,CONST_DESC,OCCUPATION_CODE,OCCUPATION_DESC,COMMUNITY_CODE,COMMUNITY_DESC,CASTE,CASTE_CODE,NATIONALITY_CODE,NATIONALITY_DESC,DOMICILE_CODE,DOMICILE_DESC,MOBILE_NO,EMAIL_ID,AADHAAR_CARD,BLOOD_GROUP,RISK_TYPE_ID,RISK_TYPE_DESC,MEMBER_FLAG,PAN,TAN,FORM_60_FLAG,CREDIT_CARD_FLAG,DEBIT_CARD_FLAG,SMS_BANKING_FLAG,MOBILE_BANKING_FLAG,NET_BANKING_FLAG,OCCUPATION_DETAILS,EMPLOYED_WITH,ANNUAL_INCOME,GUARANTEE_COUNT,SUREITY_COUNT,SUREITY_AMOUNT,DMAT_ACCNO,NRE_NRO_NRI_FLAG,PERMANENT_RETURN_DATE,IDENTIFICATION_ID,IDENTIFICATION_DETAILS,ISSUE_DATE,ISSUED_BY,ADDRESS_PROOF_ID,ADDRESS_PROOF_DETAILS,DOCUMENTS_GIVEN,KYC_COMPLETE_FLAG,KYC_COMPLETE_DATE,KYC_NUMBER,ESTABLISH_LICENCE_FLAG,ESTABLISH_LICENCE_NO,ESTABLISH_LICENCE_DATE,ESTABLISH_LICENCE_EXP,GST_NO,GST_REG_DATE,GST_VERIFY_FLAG,GST_VERIFY_DATE,OFFICE_MST_ID,OLD_PRIMARY_KEY,PERMANENT_ADD,OFFICIAL_ADD,RESISDENTIAL_ADD,PY_CUST_GEN_MST_ID,HOUSE_OWN,CAR_MOTOR_OWN,FOREGIN_TOUR_YN,CURR_LIAB_AMT,CUST_FRDT_IN_MTHS,LOAN_AC_HOLD_FRM_MTHS,SUREITY_TO_NOOFAC,CURRENT_BAL,PY_CUST_MST_ID_ALL_LOANS,PY_CUST_MST_ID_ALL,PY_CUSTCCOD_MST_ID,PY_CUSTLN_MST_ID,GLCODE,GLTITLE,NEW_AC_NUMBER,SUB_AC_NUMBER,ACCOUNT_NAME,ACCOUNT_OPEN_DATE,ACCOUNT_STATUS_DATE,ACCOUNT_TYPE,ACCOUNT_TYPE_ID,AC_SECURED_FLAG,ADDITIONAL_YN,AVG_BAL_IN_ACCOUNT,AVG_CR_IN_CASH_IN_AMOUNT,AVG_CR_IN_CLG_IN_AMOUNT,AVG_CR_IN_TRF_IN_AMOUNT,AVG_DR_IN_CASH_IN_AMOUNT,AVG_DR_IN_CLG_IN_AMOUNT,AVG_DR_IN_TRF_IN_AMOUNT,CASH_SECURITY_FLAG,DISBURSEMENT_AMOUNT,DP_AMOUNT,EMI_AMOUNT,EXPIRY_DATE,INTEREST_RATE,LAST_INTT_RECEIVED_AMT,LAST_INTT_RECEIVED_DT,LIEN_DATE,LIEN_FLAG,MISC_BALANCE,MODE_OF_OPERATION,NET_SECURITY_VALUE,NPA_CLASS_ID,NPA_DATE,NPA_FLAG,OVERDUE_AMT,OVERDUE_PRD,PENAL_AMT,PERIOD_DAYS,PERIOD_MONTHS,PERIOD_YEARS,PRINCIPAL_BALANCE,RECEIVABLE_BALANCE,RENEWAL_YN,SANCTION_AMOUNT,SANCTION_DATE,SECURED_PERCENT,SECURITY_CODE,SECURITY_VALUE,STATUS_CODE,SUBSIDY_APPLIED_FLAG,TOTAL_INTT_RECEIVED,TOTAL_NOOFTRAN_BYATM_CNT,TOTAL_NOOFTRAN_BYCASH_CNT,TOTAL_NOOFTRAN_BYCHEQUE_CNT,TOTAL_NOOFTRAN_BYCLG_CNT,TOTAL_NOOFTRAN_BYRTGS_CNT,TOTAL_NOOFTRAN_BYTRF_CNT,TOTAL_NOOF_CR_CASH_TRAN_CNT,TOTAL_NOOF_CR_CLG_TRAN_CNT,TOTAL_NOOF_CR_TRAN_CNT,TOTAL_NOOF_CR_TRF_TRAN_CNT,TOTAL_NOOF_DR_CASH_TRAN_CNT,TOTAL_NOOF_DR_CLG_TRAN_CNT,TOTAL_NOOF_DR_TRAN_CNT,TOTAL_NOOF_DR_TRF_TRAN_CNT,TOTAL_NOOF_TRAN,VALUE_DESCRIPTION,AVG_BAL_IN_ACCOUNT_SAVING_ACTIVE,AVG_BAL_IN_ACCOUNT_SAVING_CLOSED,AVG_BAL_IN_ACCOUNT_CURRENT_ACTIVE,AVG_BAL_IN_ACCOUNT_CURRENT_CLOSED,BALANCE_AMOUNT_SAVING_ACTIVE,BALANCE_AMOUNT_SAVING_CLOSED,BALANCE_AMOUNT_CURRENT_ACTIVE,BALANCE_AMOUNT_CURRENT_CLOSED,TOTAL_INTT_PAID_SAVING_ACTIVE,TOTAL_INTT_PAID_SAVING_CLOSED,TOTAL_INTT_PAID_CURRENT_ACTIVE,TOTAL_INTT_PAID_CURRENT_CLOSED,CHEQUE_BOUNCE_AMOUNT_CASA,CHEQUE_BOUNCE_CHARGES_CASA,CHEQUE_BOUNCE_CNT_CASA,DEPOSIT_AMOUNT_FDRD_ACTIVE,DEPOSIT_AMOUNT_FDRD_CLOSED,MATURITY_AMOUNT_FDRD_ACTIVE,MATURITY_AMOUNT_FDRD_CLOSED,TOTAL_PAYABLE_INTEREST_FDRD_ACTIVE,TOTAL_PAYABLE_INTEREST_FDRD_CLOSED,TOTAL_NOOF_CR_TRAN_CNT_SAVING,TOTAL_NOOF_DR_TRAN_CNT_SAVING,TOTAL_NOOF_TRAN_SAVING,TOTAL_NOOF_CR_TRAN_CNT_CURRENT,TOTAL_NOOF_DR_TRAN_CNT_CURRENT,TOTAL_NOOF_TRAN_CURRENT,TOTAL_NO_OF_ACCOUNT,TOTAL_NO_OF_LOAN_ACCOUNT,GL_TYPE,LOAN_TYPE,CHEQUE_BOUNCE_AMOUNT_CCOD,CHEQUE_BOUNCE_CHARGES_CCOD,CHEQUE_BOUNCE_CNT_CCOD
0,51,61,VAIJAPUR MERCHANTS COOPERATIVE BANK LTD,VMCBL,166,None,DATTA TRADING CO.,1996-07-03,Y,1996-07-03,M,None,NaN,N,None,N,442,0-N.A.,122,1-Individual,137,7-Other,144,0-None,None,151,445,1-Indian,443,1-Indian,None,None,None,None,447,1-Low Risk,N,None,None,0,N,N,Y,N,N,None,0,0.0,NaN,0,0.0,None,N,None,NaN,None,None,None,NaN,None,None,N,None,None,N,None,NaT,NaT,None,None,N,None,1.0,NaN,Not Specified,-,-,51,N,N,None,0.0,291,0,0,4804.48,20,88,NaN,941544.0,182,ADVANCE A

In [5]:
#copy original data for showing report in frontend

In [6]:
df2 = df.copy()
set={'1-Ssc':'SSC', '0-N.A.':'NA' ,'3-Graduation':'GRADUATION' ,'2-Hsc':'HSC' ,'4-Postgraduation':'POSTGRADUATION',
     'N':'NO', 'Y':'YES','1-Individual':'Individual',
      '2-Proprietary Firm':'Proprietary Firm', '6-Coop Society':'Coop Society', '8-Bank':'Bank',
       '7-Trust':'Trust', '3-Partnership Firm':'Partnership Firm', '10-HUF':'HUF','2-Business':'Business', 
      '7-Other':'Other', '3-House Wife':'House Wife', '4-Professional':'Professional',
       '10-AGRICULTURE / FARMER':'FARMER', '1-Service':'Service', '5-Retired':'Retired', '11-DOCTOR':'DOCTOR',
       '9-PRIVATE EMPLOYEE':'PRIVATE EMPLOYEE', '12-ADVOCATE':'ADVOCATE', '6-Student':'Student',
       '8-GOVT. EMPLOYEE':'GOVT EMPLOYEE', '17-PENSHIONER':'PENSHIONER',
       '13-ENGINEER':'ENGINEER', '19-UNEMPLOYED':'UNEMPLOYED',  '14-TRADING':'TRADING',
       '18-EX-SERVICE MAN':'EXSERVICE MAN', '16-COMMISSION AGENT':'COMMISSION AGENT','1-Hindu':'Hindu', '0-None':'None', '3-Muslim':'Muslim', 
      '2-Sikh':'Sikh', '6-Buddhists':'Buddhists','4-Christian':'Christian','1-Albanians':'Albanians', '82-Indians':'Indians',
      '77-Guyanese':'Guyanese', '59-Faroese':'Faroese',
       '87-Israelis':'Israelis', '174-Vanuatuans':'Vanuatuans', '111-Manx':'Manx','1-Indian':'Indian', '2-Others':'Others',
       '3-High Risk':'High Risk', '1-Low Risk':'Low Risk', '2-Medium Risk':'Medium Risk','M':'Manualy'}
set1 = {'M':'Male','F':'Female','N':'Not Specified'}
set2 = {'M':'Married', 'N':'Not Specified', 'U':'UnMarried', 'W':'Widow', 'D':'Divorce'}
set3 = {166:'General Customer',168:'Firm', 173:'Senior Citizen', 
      174:'Society',175:'Staff',177:'Student',178:'Trust',
        179:'Women',574:'Handicap',1414:'NA',181:'Self',182:'Proprietor', 
      183:'Either Or Survivor', 184:'Former Or Survivor',185:'Jointly Or Severally',186:'Anyone Single Or Survivor',
      187:'Any Two Jointly',189:'All Jointly',190:'Only First',192:'No 1,2,3 Jointly',197:'Authorised Signatory',
      199:'Managing Director',207:'Minor And Natural Guardian',208:'Minor And Legal Guardian',209:'Minor Alone',
      912:'NA',932:'NA'}
set4 = {'N':'NOT APPLI', 'S':'SHARE HOLDER', 'L':'NOMINAL MEMBER','C':'Closed', 'A':'Active', 'H':'Hold', 'D':'Defreezed',
         'T':'Amountwise freezed', 'F':'Freezed'}
df2_cat = df2.copy()
for x in df2_cat:
    if x =='CUSTOMER_GENDER':
        df2_cat[x]=df2_cat[x].map(set1)
    elif x =='MARTIAL_STATUS':
        df2_cat[x]=df2_cat[x].map(set2)
    elif x =='CUSTOMER_TYPE':
        df2_cat[x]=df2_cat[x].map(set3)
    elif x =='MODE_OF_OPERATION':
        df2_cat[x]=df2_cat[x].map(set3) 
    elif x =='MEMBER_FLAG':
        df2_cat[x]=df2_cat[x].map(set4)            
    elif x =='STATUS_CODE':
        df2_cat[x]=df2_cat[x].map(set4) 
    else:    
        df2_cat[x] = df2_cat[x].map(set)
df2_cat = df2_cat.dropna(axis=1,how='all')
ss = df2_cat.columns
df2[ss]=df2_cat[ss]
df2

,PY_CUSTOMER_ID,CUSTOMER_ID,CUSTOMER_BANK_NAME,CUSTOMER_BANK_SHORT_NAME,CUSTOMER_TYPE,CUSTOMER_TITLE,CUSTOMER_NAME,CUSTOMER_CREATION_DATE,STATUS_CODE_MST,CUSTOMER_STATUS_DATE,CUST_CREATE_FROM,DATE_OF_BIRTH,CUSTOMER_AGE,CUSTOMER_GENDER,CUST_BIRTH_PLACE,MARTIAL_STATUS,EDUCATION_ID,EDUCATION_DESC,CONST_CODE,CONST_DESC,OCCUPATION_CODE,OCCUPATION_DESC,COMMUNITY_CODE,COMMUNITY_DESC,CASTE,CASTE_CODE,NATIONALITY_CODE,NATIONALITY_DESC,DOMICILE_CODE,DOMICILE_DESC,MOBILE_NO,EMAIL_ID,AADHAAR_CARD,BLOOD_GROUP,RISK_TYPE_ID,RISK_TYPE_DESC,MEMBER_FLAG,PAN,TAN,FORM_60_FLAG,CREDIT_CARD_FLAG,DEBIT_CARD_FLAG,SMS_BANKING_FLAG,MOBILE_BANKING_FLAG,NET_BANKING_FLAG,OCCUPATION_DETAILS,EMPLOYED_WITH,ANNUAL_INCOME,GUARANTEE_COUNT,SUREITY_COUNT,SUREITY_AMOUNT,DMAT_ACCNO,NRE_NRO_NRI_FLAG,PERMANENT_RETURN_DATE,IDENTIFICATION_ID,IDENTIFICATION_DETAILS,ISSUE_DATE,ISSUED_BY,ADDRESS_PROOF_ID,ADDRESS_PROOF_DETAILS,DOCUMENTS_GIVEN,KYC_COMPLETE_FLAG,KYC_COMPLETE_DATE,KYC_NUMBER,ESTABLISH_LICENCE_FLAG,ESTABLISH_LICENCE_NO,ESTABLISH_LICENCE_DATE,ESTABLISH_LICENCE_EXP,GST_NO,GST_REG_DATE,GST_VERIFY_FLAG,GST_VERIFY_DATE,OFFICE_MST_ID,OLD_PRIMARY_KEY,PERMANENT_ADD,OFFICIAL_ADD,RESISDENTIAL_ADD,PY_CUST_GEN_MST_ID,HOUSE_OWN,CAR_MOTOR_OWN,FOREGIN_TOUR_YN,CURR_LIAB_AMT,CUST_FRDT_IN_MTHS,LOAN_AC_HOLD_FRM_MTHS,SUREITY_TO_NOOFAC,CURRENT_BAL,PY_CUST_MST_ID_ALL_LOANS,PY_CUST_MST_ID_ALL,PY_CUSTCCOD_MST_ID,PY_CUSTLN_MST_ID,GLCODE,GLTITLE,NEW_AC_NUMBER,SUB_AC_NUMBER,ACCOUNT_NAME,ACCOUNT_OPEN_DATE,ACCOUNT_STATUS_DATE,ACCOUNT_TYPE,ACCOUNT_TYPE_ID,AC_SECURED_FLAG,ADDITIONAL_YN,AVG_BAL_IN_ACCOUNT,AVG_CR_IN_CASH_IN_AMOUNT,AVG_CR_IN_CLG_IN_AMOUNT,AVG_CR_IN_TRF_IN_AMOUNT,AVG_DR_IN_CASH_IN_AMOUNT,AVG_DR_IN_CLG_IN_AMOUNT,AVG_DR_IN_TRF_IN_AMOUNT,CASH_SECURITY_FLAG,DISBURSEMENT_AMOUNT,DP_AMOUNT,EMI_AMOUNT,EXPIRY_DATE,INTEREST_RATE,LAST_INTT_RECEIVED_AMT,LAST_INTT_RECEIVED_DT,LIEN_DATE,LIEN_FLAG,MISC_BALANCE,MODE_OF_OPERATION,NET_SECURITY_VALUE,NPA_CLASS_ID,NPA_DATE,NPA_FLAG,OVERDUE_AMT,OVERDUE_PRD,PENAL_AMT,PERIOD_DAYS,PERIOD_MONTHS,PERIOD_YEARS,PRINCIPAL_BALANCE,RECEIVABLE_BALANCE,RENEWAL_YN,SANCTION_AMOUNT,SANCTION_DATE,SECURED_PERCENT,SECURITY_CODE,SECURITY_VALUE,STATUS_CODE,SUBSIDY_APPLIED_FLAG,TOTAL_INTT_RECEIVED,TOTAL_NOOFTRAN_BYATM_CNT,TOTAL_NOOFTRAN_BYCASH_CNT,TOTAL_NOOFTRAN_BYCHEQUE_CNT,TOTAL_NOOFTRAN_BYCLG_CNT,TOTAL_NOOFTRAN_BYRTGS_CNT,TOTAL_NOOFTRAN_BYTRF_CNT,TOTAL_NOOF_CR_CASH_TRAN_CNT,TOTAL_NOOF_CR_CLG_TRAN_CNT,TOTAL_NOOF_CR_TRAN_CNT,TOTAL_NOOF_CR_TRF_TRAN_CNT,TOTAL_NOOF_DR_CASH_TRAN_CNT,TOTAL_NOOF_DR_CLG_TRAN_CNT,TOTAL_NOOF_DR_TRAN_CNT,TOTAL_NOOF_DR_TRF_TRAN_CNT,TOTAL_NOOF_TRAN,VALUE_DESCRIPTION,AVG_BAL_IN_ACCOUNT_SAVING_ACTIVE,AVG_BAL_IN_ACCOUNT_SAVING_CLOSED,AVG_BAL_IN_ACCOUNT_CURRENT_ACTIVE,AVG_BAL_IN_ACCOUNT_CURRENT_CLOSED,BALANCE_AMOUNT_SAVING_ACTIVE,BALANCE_AMOUNT_SAVING_CLOSED,BALANCE_AMOUNT_CURRENT_ACTIVE,BALANCE_AMOUNT_CURRENT_CLOSED,TOTAL_INTT_PAID_SAVING_ACTIVE,TOTAL_INTT_PAID_SAVING_CLOSED,TOTAL_INTT_PAID_CURRENT_ACTIVE,TOTAL_INTT_PAID_CURRENT_CLOSED,CHEQUE_BOUNCE_AMOUNT_CASA,CHEQUE_BOUNCE_CHARGES_CASA,CHEQUE_BOUNCE_CNT_CASA,DEPOSIT_AMOUNT_FDRD_ACTIVE,DEPOSIT_AMOUNT_FDRD_CLOSED,MATURITY_AMOUNT_FDRD_ACTIVE,MATURITY_AMOUNT_FDRD_CLOSED,TOTAL_PAYABLE_INTEREST_FDRD_ACTIVE,TOTAL_PAYABLE_INTEREST_FDRD_CLOSED,TOTAL_NOOF_CR_TRAN_CNT_SAVING,TOTAL_NOOF_DR_TRAN_CNT_SAVING,TOTAL_NOOF_TRAN_SAVING,TOTAL_NOOF_CR_TRAN_CNT_CURRENT,TOTAL_NOOF_DR_TRAN_CNT_CURRENT,TOTAL_NOOF_TRAN_CURRENT,TOTAL_NO_OF_ACCOUNT,TOTAL_NO_OF_LOAN_ACCOUNT,GL_TYPE,LOAN_TYPE,CHEQUE_BOUNCE_AMOUNT_CCOD,CHEQUE_BOUNCE_CHARGES_CCOD,CHEQUE_BOUNCE_CNT_CCOD
0,51,61,VAIJAPUR MERCHANTS COOPERATIVE BANK LTD,VMCBL,General Customer,None,DATTA TRADING CO.,1996-07-03,YES,1996-07-03,Manualy,None,NaN,Not Specified,None,Not Specified,442,NA,122,Individual,137,Other,144,None,NaN,151,445,Indian,443,Indian,None,None,None,None,447,Low Risk,NOT APPLI,None,None,0,NO,NO,YES,NO,NO,None,0,0.0,NaN,0,0.0,None,NO,None,NaN,NaN,None,NaN,NaN,None,None,NO,None,None,NO,None,NaT,NaT,None,None,NO,None,1.0,NaN,Not Specified,-,-,51,NO,NO,None,0.0,29

In [7]:
#Drop the features for machine learning 

In [8]:
df= df.drop(['CUSTOMER_ID','CUSTOMER_BANK_NAME','CUSTOMER_CREATION_DATE','CUSTOMER_TITLE','CUSTOMER_NAME','STATUS_CODE_MST',
             'CUSTOMER_STATUS_DATE','DATE_OF_BIRTH','CUST_BIRTH_PLACE','EDUCATION_DESC','CONST_DESC',
        'OCCUPATION_DESC','COMMUNITY_DESC','CASTE','NATIONALITY_DESC','DOMICILE_DESC','EMAIL_ID','RISK_TYPE_DESC','TAN',
             'OCCUPATION_DETAILS','EMPLOYED_WITH','DMAT_ACCNO','PERMANENT_RETURN_DATE','IDENTIFICATION_DETAILS','ISSUE_DATE',
        'ISSUED_BY','ADDRESS_PROOF_DETAILS','DOCUMENTS_GIVEN','KYC_COMPLETE_DATE','KYC_NUMBER','ESTABLISH_LICENCE_NO',
             'ESTABLISH_LICENCE_DATE','ESTABLISH_LICENCE_EXP','GST_NO','GST_REG_DATE','GST_VERIFY_DATE','OFFICE_MST_ID',
            'OLD_PRIMARY_KEY','OFFICIAL_ADD','RESISDENTIAL_ADD','PERMANENT_ADD','PY_CUST_GEN_MST_ID',
        'FOREGIN_TOUR_YN','CUST_FRDT_IN_MTHS','SUREITY_TO_NOOFAC', 'PY_CUST_MST_ID_ALL_LOANS','PY_CUST_MST_ID_ALL',
        'PY_CUSTCCOD_MST_ID','PY_CUSTLN_MST_ID','GLTITLE','NEW_AC_NUMBER','ACCOUNT_NAME','ACCOUNT_OPEN_DATE',
        'ACCOUNT_STATUS_DATE','DP_AMOUNT','EXPIRY_DATE','LAST_INTT_RECEIVED_DT','LIEN_DATE','NPA_DATE',
            'SANCTION_DATE','SECURITY_CODE','VALUE_DESCRIPTION','LOAN_TYPE','BLOOD_GROUP','GLCODE'],axis = 1)
df

,PY_CUSTOMER_ID,CUSTOMER_BANK_SHORT_NAME,CUSTOMER_TYPE,CUST_CREATE_FROM,CUSTOMER_AGE,CUSTOMER_GENDER,MARTIAL_STATUS,EDUCATION_ID,CONST_CODE,OCCUPATION_CODE,COMMUNITY_CODE,CASTE_CODE,NATIONALITY_CODE,DOMICILE_CODE,MOBILE_NO,AADHAAR_CARD,RISK_TYPE_ID,MEMBER_FLAG,PAN,FORM_60_FLAG,CREDIT_CARD_FLAG,DEBIT_CARD_FLAG,SMS_BANKING_FLAG,MOBILE_BANKING_FLAG,NET_BANKING_FLAG,ANNUAL_INCOME,GUARANTEE_COUNT,SUREITY_COUNT,SUREITY_AMOUNT,NRE_NRO_NRI_FLAG,IDENTIFICATION_ID,ADDRESS_PROOF_ID,KYC_COMPLETE_FLAG,ESTABLISH_LICENCE_FLAG,GST_VERIFY_FLAG,HOUSE_OWN,CAR_MOTOR_OWN,CURR_LIAB_AMT,LOAN_AC_HOLD_FRM_MTHS,CURRENT_BAL,SUB_AC_NUMBER,ACCOUNT_TYPE,ACCOUNT_TYPE_ID,AC_SECURED_FLAG,ADDITIONAL_YN,AVG_BAL_IN_ACCOUNT,AVG_CR_IN_CASH_IN_AMOUNT,AVG_CR_IN_CLG_IN_AMOUNT,AVG_CR_IN_TRF_IN_AMOUNT,AVG_DR_IN_CASH_IN_AMOUNT,AVG_DR_IN_CLG_IN_AMOUNT,AVG_DR_IN_TRF_IN_AMOUNT,CASH_SECURITY_FLAG,DISBURSEMENT_AMOUNT,EMI_AMOUNT,INTEREST_RATE,LAST_INTT_RECEIVED_AMT,LIEN_FLAG,MISC_BALANCE,MODE_OF_OPERATION,NET_SECURITY_VALUE,NPA_CLASS_ID,NPA_FLAG,OVERDUE_AMT,OVERDUE_PRD,PENAL_AMT,PERIOD_DAYS,PERIOD_MONTHS,PERIOD_YEARS,PRINCIPAL_BALANCE,RECEIVABLE_BALANCE,RENEWAL_YN,SANCTION_AMOUNT,SECURED_PERCENT,SECURITY_VALUE,STATUS_CODE,SUBSIDY_APPLIED_FLAG,TOTAL_INTT_RECEIVED,TOTAL_NOOFTRAN_BYATM_CNT,TOTAL_NOOFTRAN_BYCASH_CNT,TOTAL_NOOFTRAN_BYCHEQUE_CNT,TOTAL_NOOFTRAN_BYCLG_CNT,TOTAL_NOOFTRAN_BYRTGS_CNT,TOTAL_NOOFTRAN_BYTRF_CNT,TOTAL_NOOF_CR_CASH_TRAN_CNT,TOTAL_NOOF_CR_CLG_TRAN_CNT,TOTAL_NOOF_CR_TRAN_CNT,TOTAL_NOOF_CR_TRF_TRAN_CNT,TOTAL_NOOF_DR_CASH_TRAN_CNT,TOTAL_NOOF_DR_CLG_TRAN_CNT,TOTAL_NOOF_DR_TRAN_CNT,TOTAL_NOOF_DR_TRF_TRAN_CNT,TOTAL_NOOF_TRAN,AVG_BAL_IN_ACCOUNT_SAVING_ACTIVE,AVG_BAL_IN_ACCOUNT_SAVING_CLOSED,AVG_BAL_IN_ACCOUNT_CURRENT_ACTIVE,AVG_BAL_IN_ACCOUNT_CURRENT_CLOSED,BALANCE_AMOUNT_SAVING_ACTIVE,BALANCE_AMOUNT_SAVING_CLOSED,BALANCE_AMOUNT_CURRENT_ACTIVE,BALANCE_AMOUNT_CURRENT_CLOSED,TOTAL_INTT_PAID_SAVING_ACTIVE,TOTAL_INTT_PAID_SAVING_CLOSED,TOTAL_INTT_PAID_CURRENT_ACTIVE,TOTAL_INTT_PAID_CURRENT_CLOSED,CHEQUE_BOUNCE_AMOUNT_CASA,CHEQUE_BOUNCE_CHARGES_CASA,CHEQUE_BOUNCE_CNT_CASA,DEPOSIT_AMOUNT_FDRD_ACTIVE,DEPOSIT_AMOUNT_FDRD_CLOSED,MATURITY_AMOUNT_FDRD_ACTIVE,MATURITY_AMOUNT_FDRD_CLOSED,TOTAL_PAYABLE_INTEREST_FDRD_ACTIVE,TOTAL_PAYABLE_INTEREST_FDRD_CLOSED,TOTAL_NOOF_CR_TRAN_CNT_SAVING,TOTAL_NOOF_DR_TRAN_CNT_SAVING,TOTAL_NOOF_TRAN_SAVING,TOTAL_NOOF_CR_TRAN_CNT_CURRENT,TOTAL_NOOF_DR_TRAN_CNT_CURRENT,TOTAL_NOOF_TRAN_CURRENT,TOTAL_NO_OF_ACCOUNT,TOTAL_NO_OF_LOAN_ACCOUNT,GL_TYPE,CHEQUE_BOUNCE_AMOUNT_CCOD,CHEQUE_BOUNCE_CHARGES_CCOD,CHEQUE_BOUNCE_CNT_CCOD
0,51,VMCBL,166,M,NaN,N,N,442,122,137,144,151,445,443,None,None,447,N,None,0,N,N,Y,N,N,0.0,NaN,0,0.0,N,NaN,NaN,N,N,N,N,N,0.0,0,4804.48,1,General Customer,449,N,N,0.00,0.00,0.0,0.0,0.0,0.00,0.00,N,185000.0,74644.0,15.0,0.0,N,0.0,181,0.0,ST,N,0.0,0.0,0.0,16,2,0,0.0,0.0,N,185000.0,0.0,313200.0,C,N,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,164710.44,0.00,NaN,0.0,4804.48,0.00,NaN,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,18,18,2,1,Term Loan,0.0,0.0,0
1,58,VMCBL,168,M,NaN,N,N,442,123,137,144,151,445,443,9422208499,None,447,N,ABOPB6966H,2,N,N,Y,Y,N,0.0,0.0,0,0.0,N,437.0,NaN,Y,N,N,N,N,-45910.6,0,28828.09,1,Firm,449,Y,N,-187994.71,587480.77,8550.0,99500.0,0.0,305947.25,184616.89,N,NaN,NaN,14.0,809.0,N,0.0,181,6130000.0,ST,N,0.0,0.0,0.0,0,36,0,-45910.6,0.0,N,500000.0,0.0,6130000.0,C,N,809.0,0,54,36,45,20,56,54,3,58,1,0,42,122,80,180,NaN,0.0,0.00,316463.91,NaN,0.0,0.00,28828.09,NaN,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,92,54,146,2,1,None,0.0,0.0,0
2,100,VMCBL,166,M,NaN,N,N,442,122,137,144,151,445,443,0,None,447,N,None,0,N,N,Y,N,N,0.0,NaN,0,0.0,N,NaN,NaN,N,N,N,N,N,0.0,0,0.00,1,General Customer,449,N,N,0.00,0.00,0.0,0.0,0.0,0.00,0.00,None,NaN,NaN,0.0,0.0,N,0.0,181,0.0,ST,N,0.0,0.0,0.0,0,3,0,0.0,0.0,N,10000.0,0.0,0.0,C,N,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,0.00,0.00,NaN,0.0,0.00,0.00,NaN,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,1,None,0.0,0.0,0
3,107,VMCBL,168,M,59.0,N,N,442,123,137,144,151,445,443,

In [9]:
#Check missing data & cleaning the data using missing value treatment

In [10]:
df_cat1 = df.select_dtypes(include = 'object').copy()
cols = list(df_cat1.columns)
for columns in cols:
    df_cat1[columns] = df_cat1[columns].apply(lambda x: 'YES' if len(str(x)) == 10 or len(str(x)) == 12  else 'NO')
sd=list(df_cat1.columns[(df_cat1 == 'YES').any(axis=0)])
df[sd]=df_cat1[sd]

df_cat = df.select_dtypes(include = 'object').copy()
df_num = df.select_dtypes(include = 'number').copy()
df_num.fillna(0, inplace=True)
df_cat.fillna(df_cat.mode().iloc[0], inplace=True)
cols = list(df_num.columns)
for col in cols:
    df_num[col] = pd.to_numeric(df_num[col].map(lambda x: str(x).lstrip('-').rstrip('-')))
df = pd.concat([df_num,df_cat],axis = 1)
df

,PY_CUSTOMER_ID,CUSTOMER_TYPE,CUSTOMER_AGE,EDUCATION_ID,CONST_CODE,OCCUPATION_CODE,COMMUNITY_CODE,CASTE_CODE,NATIONALITY_CODE,DOMICILE_CODE,RISK_TYPE_ID,ANNUAL_INCOME,GUARANTEE_COUNT,SUREITY_COUNT,SUREITY_AMOUNT,IDENTIFICATION_ID,ADDRESS_PROOF_ID,CURR_LIAB_AMT,LOAN_AC_HOLD_FRM_MTHS,CURRENT_BAL,SUB_AC_NUMBER,ACCOUNT_TYPE_ID,AVG_BAL_IN_ACCOUNT,AVG_CR_IN_CASH_IN_AMOUNT,AVG_CR_IN_CLG_IN_AMOUNT,AVG_CR_IN_TRF_IN_AMOUNT,AVG_DR_IN_CASH_IN_AMOUNT,AVG_DR_IN_CLG_IN_AMOUNT,AVG_DR_IN_TRF_IN_AMOUNT,DISBURSEMENT_AMOUNT,EMI_AMOUNT,INTEREST_RATE,LAST_INTT_RECEIVED_AMT,MISC_BALANCE,MODE_OF_OPERATION,NET_SECURITY_VALUE,OVERDUE_AMT,OVERDUE_PRD,PENAL_AMT,PERIOD_DAYS,PERIOD_MONTHS,PERIOD_YEARS,PRINCIPAL_BALANCE,RECEIVABLE_BALANCE,SANCTION_AMOUNT,SECURED_PERCENT,SECURITY_VALUE,TOTAL_INTT_RECEIVED,TOTAL_NOOFTRAN_BYATM_CNT,TOTAL_NOOFTRAN_BYCASH_CNT,TOTAL_NOOFTRAN_BYCHEQUE_CNT,TOTAL_NOOFTRAN_BYCLG_CNT,TOTAL_NOOFTRAN_BYRTGS_CNT,TOTAL_NOOFTRAN_BYTRF_CNT,TOTAL_NOOF_CR_CASH_TRAN_CNT,TOTAL_NOOF_CR_CLG_TRAN_CNT,TOTAL_NOOF_CR_TRAN_CNT,TOTAL_NOOF_CR_TRF_TRAN_CNT,TOTAL_NOOF_DR_CASH_TRAN_CNT,TOTAL_NOOF_DR_CLG_TRAN_CNT,TOTAL_NOOF_DR_TRAN_CNT,TOTAL_NOOF_DR_TRF_TRAN_CNT,TOTAL_NOOF_TRAN,AVG_BAL_IN_ACCOUNT_SAVING_ACTIVE,AVG_BAL_IN_ACCOUNT_SAVING_CLOSED,AVG_BAL_IN_ACCOUNT_CURRENT_ACTIVE,AVG_BAL_IN_ACCOUNT_CURRENT_CLOSED,BALANCE_AMOUNT_SAVING_ACTIVE,BALANCE_AMOUNT_SAVING_CLOSED,BALANCE_AMOUNT_CURRENT_ACTIVE,BALANCE_AMOUNT_CURRENT_CLOSED,TOTAL_INTT_PAID_SAVING_ACTIVE,TOTAL_INTT_PAID_SAVING_CLOSED,TOTAL_INTT_PAID_CURRENT_ACTIVE,TOTAL_INTT_PAID_CURRENT_CLOSED,CHEQUE_BOUNCE_AMOUNT_CASA,CHEQUE_BOUNCE_CHARGES_CASA,CHEQUE_BOUNCE_CNT_CASA,DEPOSIT_AMOUNT_FDRD_ACTIVE,DEPOSIT_AMOUNT_FDRD_CLOSED,MATURITY_AMOUNT_FDRD_ACTIVE,MATURITY_AMOUNT_FDRD_CLOSED,TOTAL_PAYABLE_INTEREST_FDRD_ACTIVE,TOTAL_PAYABLE_INTEREST_FDRD_CLOSED,TOTAL_NOOF_CR_TRAN_CNT_SAVING,TOTAL_NOOF_DR_TRAN_CNT_SAVING,TOTAL_NOOF_TRAN_SAVING,TOTAL_NOOF_CR_TRAN_CNT_CURRENT,TOTAL_NOOF_DR_TRAN_CNT_CURRENT,TOTAL_NOOF_TRAN_CURRENT,TOTAL_NO_OF_ACCOUNT,TOTAL_NO_OF_LOAN_ACCOUNT,CHEQUE_BOUNCE_AMOUNT_CCOD,CHEQUE_BOUNCE_CHARGES_CCOD,CHEQUE_BOUNCE_CNT_CCOD,CUSTOMER_BANK_SHORT_NAME,CUST_CREATE_FROM,CUSTOMER_GENDER,MARTIAL_STATUS,MOBILE_NO,AADHAAR_CARD,MEMBER_FLAG,PAN,FORM_60_FLAG,CREDIT_CARD_FLAG,DEBIT_CARD_FLAG,SMS_BANKING_FLAG,MOBILE_BANKING_FLAG,NET_BANKING_FLAG,NRE_NRO_NRI_FLAG,KYC_COMPLETE_FLAG,ESTABLISH_LICENCE_FLAG,GST_VERIFY_FLAG,HOUSE_OWN,CAR_MOTOR_OWN,ACCOUNT_TYPE,AC_SECURED_FLAG,ADDITIONAL_YN,CASH_SECURITY_FLAG,LIEN_FLAG,NPA_CLASS_ID,NPA_FLAG,RENEWAL_YN,STATUS_CODE,SUBSIDY_APPLIED_FLAG,GL_TYPE
0,51,166,0.0,442,122,137,144,151,445,443,447,0.0,0.0,0,0.0,0.0,0.0,0.0,0,4804.48,1,449,0.00,0.00,0.0,0.0,0.0,0.00,0.00,185000.0,74644.0,15.0,0.0,0.0,181,0.0,0.0,0.0,0.0,16,2,0,0.0,0.0,185000.0,0.0,313200.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,164710.44,0.00,0.0,0.0,4804.48,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,18,18,2,1,0.0,0.0,0,VMCBL,M,N,N,NO,NO,N,NO,0,N,N,Y,N,N,N,N,N,N,N,N,General Customer,N,N,N,N,ST,N,N,C,N,Term Loan
1,58,168,0.0,442,123,137,144,151,445,443,447,0.0,0.0,0,0.0,437.0,0.0,45910.6,0,28828.09,1,449,187994.71,587480.77,8550.0,99500.0,0.0,305947.25,184616.89,0.0,0.0,14.0,809.0,0.0,181,6130000.0,0.0,0.0,0.0,0,36,0,45910.6,0.0,500000.0,0.0,6130000.0,809.0,0,54,36,45,20,56,54,3,58,1,0,42,122,80,180,0.0,0.0,0.00,316463.91,0.0,0.0,0.00,28828.09,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,92,54,146,2,1,0.0,0.0,0,VMCBL,M,N,N,YES,NO,N,YES,2,N,N,Y,Y,N,N,Y,N,N,N,N,Firm,Y,N,N,N,ST,N,N,C,N,Term Loan
2,100,166,0.0,442,122,137,144,151,445,443,447,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.00,1,449,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,181,0.0,0.0,0.0,0.0,0,3,0,0.0,0.0,10000.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,1,0.0,0.0,0,VMCBL,M,N,N,NO,NO,N,NO,0,N,N,Y,N,N,N,N,N,N,N,N,General Customer,N,N,N,N,ST,N,N,C,N,Term Loan
3,107,168,59.0,442,123,137,144,151,445,443,447,0.0,0.0,1,500000.0,945.0,0.0,

In [11]:
#create Target variabale and divided data in training and testing

In [12]:
df['good_bad'] = np.where(df.loc[:, 'NPA_CLASS_ID'].isin(['ST','SS']), 1, 0)
# Drop the original column
df.drop(columns = ['NPA_CLASS_ID'], inplace = True)
X = df.drop('good_bad', axis = 1)
y = df['good_bad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100, stratify = y)

In [13]:
#Lebal Encoding for NPA generations

In [14]:
from sklearn.preprocessing import LabelEncoder
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
df_cat = df.select_dtypes(include = 'object').copy()    
cols = list(df_cat.columns)
X1=MultiColumnLabelEncoder(columns = cols).fit_transform(X)
X_train1=MultiColumnLabelEncoder(columns = cols).fit_transform(X_train)
X_test1=MultiColumnLabelEncoder(columns = cols).fit_transform(X_test)

In [15]:
#Calculate Chi-square value and select only p-value of chi square is zero

In [16]:
chi2_check = {}
# loop over each column in the training set to calculate chi-statistic with the categorical target variable
for column in df_cat:
    chi, p, dof, ex = chi2_contingency(pd.crosstab(y_train, df_cat[column]))
    chi2_check.setdefault('Feature',[]).append(column)
    chi2_check.setdefault('p-value',[]).append(round(p, 10))
# convert the dictionary to a DF
chi2_result = pd.DataFrame(data = chi2_check)
chi2_result.sort_values(by = ['p-value'], ascending = True, inplace = True)
chi = chi2_result.reset_index(drop=True)
chi = chi[chi['p-value']== 000000e+00]
chi = list(chi['Feature'])
chi

['DEBIT_CARD_FLAG',
 'CASH_SECURITY_FLAG',
 'KYC_COMPLETE_FLAG',
 'LIEN_FLAG',
 'NPA_FLAG',
 'PAN',
 'AC_SECURED_FLAG',
 'AADHAAR_CARD',
 'MOBILE_NO',
 'MARTIAL_STATUS',
 'CUSTOMER_GENDER',
 'STATUS_CODE']

In [17]:
# function to create dummy variables on categorical variable where p value zero

In [18]:
def dummy_creation(df, columns_list):
    df_dummies = []
    for col in columns_list:
        df_dummies.append(pd.get_dummies(df[col], prefix = col, prefix_sep = ':'))
    df_dummies = pd.concat(df_dummies, axis = 1)
    df = pd.concat([df, df_dummies], axis = 1)
    return df
# apply to our final  categorical variables
X_train = dummy_creation(X_train, chi)

X_test = dummy_creation(X_test, chi)

X2 = df.drop('good_bad', axis = 1)
X_dummy = dummy_creation(X2, chi)
X4 = dummy_creation(X, chi)
# reindex the dummied test set variables to make sure all the feature columns in the train set are also available in the test set
X3 = X4.reindex(labels=X4.columns, axis=1, fill_value=0)
X_test = X_test.reindex(labels=X_train.columns, axis=1, fill_value=0)


In [19]:
# Enter input Py_customer_id for check the output on individual ID
py_customer_id=int(input('Enter the customer id: '))
X1 = X1.loc[X1['PY_CUSTOMER_ID'] == py_customer_id]
X12 = X3.copy()
X12 = X12.loc[X12['PY_CUSTOMER_ID'] == py_customer_id]

Enter the customer id: 3


In [20]:
# Calculate F Statistic and corresponding p values on numerical data

In [21]:
F_statistic, p_values = f_classif(df_num, y)
# convert to a DF
ANOVA_F_table = pd.DataFrame(data = {'Numerical_Feature': df_num.columns.values, 'F-Score': F_statistic, 'p values': p_values.round(decimals=10)})
ANOVA_F_table.sort_values(by = ['F-Score'], ascending = False, inplace = True)
ANOVA_F_table = ANOVA_F_table.reset_index(drop=True)
anova = ANOVA_F_table[ANOVA_F_table['p values']==000000e+00]
anova = list(anova['Numerical_Feature'])
anova

C:\Users\vishal.lote\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [12 18 33 43 48 73 74 76 93] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\vishal.lote\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


['OVERDUE_PRD',
 'PERIOD_MONTHS',
 'RISK_TYPE_ID',
 'INTEREST_RATE',
 'PERIOD_YEARS',
 'TOTAL_NO_OF_LOAN_ACCOUNT',
 'AVG_BAL_IN_ACCOUNT',
 'PERIOD_DAYS',
 'TOTAL_NO_OF_ACCOUNT',
 'OVERDUE_AMT',
 'IDENTIFICATION_ID',
 'TOTAL_NOOF_DR_TRAN_CNT_SAVING',
 'TOTAL_NOOF_DR_TRF_TRAN_CNT']

In [22]:
# We define a function to calculate WoE of continuous variables

In [23]:
def woe_ordered_continuous(df, continuous_variabe_name, y_df):
    df = pd.concat([df[continuous_variabe_name], y_df], axis = 1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                    df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    #df = df.sort_values(['WoE'])
    #df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df

In [24]:
# Create copies of the 4 training sets to be preprocessed using WoE
X_train_prepr = X_train.copy()
y_train_prepr = y_train.copy()
X_test_prepr = X_test.copy()
y_test_prepr = y_test.copy()

In [25]:
#Calculate woe and iv value table ranges of anova features on numerical data 

In [26]:
df_temps = []
df_temp3 = []
for i in range (len(anova)):
    X_train_prepr[anova[i]] = pd.cut(X_train_prepr[anova[i]], 25)
    df_temp = woe_ordered_continuous(X_train_prepr, anova[i], y_train_prepr)
    df_temp = df_temp.replace([np.inf, -np.inf], np.nan)
    df_temp=df_temp[df_temp.WoE.notnull()].reset_index()
    df_temp=pd.DataFrame(df_temp[anova[i]])
    df_temp[anova[i]] = df_temp[anova[i]].map(lambda x: str(x).lstrip('(').rstrip(']'))
    df_temps.append(df_temp)
    df_temp2=pd.get_dummies(df_temp,columns =[anova[i]], prefix = anova[i], prefix_sep = ':')
    ss=list(df_temp2.columns[(df_temp2 == 1).any(axis=0) ])
    df_temp3.append(df_temp2[ss].columns)
df_temp3

C:\Users\vishal.lote\Anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\vishal.lote\Anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\vishal.lote\Anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[Index(['OVERDUE_PRD:-0.161, 6.435', 'OVERDUE_PRD:6.435, 12.87',
        'OVERDUE_PRD:12.87, 19.304', 'OVERDUE_PRD:19.304, 25.739',
        'OVERDUE_PRD:25.739, 32.174', 'OVERDUE_PRD:32.174, 38.609',
        'OVERDUE_PRD:38.609, 45.044', 'OVERDUE_PRD:45.044, 51.478',
        'OVERDUE_PRD:51.478, 57.913', 'OVERDUE_PRD:57.913, 64.348',
        'OVERDUE_PRD:64.348, 70.783', 'OVERDUE_PRD:70.783, 77.218',
        'OVERDUE_PRD:83.652, 90.087'],
       dtype='object'),
 Index(['PERIOD_MONTHS:-0.999, 39.96', 'PERIOD_MONTHS:39.96, 79.92',
        'PERIOD_MONTHS:79.92, 119.88', 'PERIOD_MONTHS:119.88, 159.84',
        'PERIOD_MONTHS:159.84, 199.8'],
       dtype='object'),
 Index(['RISK_TYPE_ID:446.766, 456.36', 'RISK_TYPE_ID:671.64, 681.0'], dtype='object'),
 Index(['INTEREST_RATE:7.92, 8.64', 'INTEREST_RATE:9.36, 10.08',
        'INTEREST_RATE:10.8, 11.52', 'INTEREST_RATE:11.52, 12.24',
        'INTEREST_RATE:12.96, 13.68', 'INTEREST_RATE:13.68, 14.4',
        'INTEREST_RATE:14.4, 15.12', 'INTE

In [27]:
#Define Custom class for WoE Binning

In [28]:
my_list = list(itertools.chain(*df_temp3))
cols = my_list
cols2 = pd.DataFrame(cols)
cols1 = cols2[0].str.split(':').str[0]        
mylist = cols2[0].str.split(':').str[1]
cols_1 = list(X_dummy.drop(X_dummy.iloc[:, 0:len(X2.columns)].columns, axis = 1) )

class WoE_Binning(BaseEstimator, TransformerMixin):
    def __init__(self, X): # no *args or *kargs
        self.X = X
    def fit(self, X, y = None):
        return self #nothing else to do
    def transform(self, X):
        X_new = X.loc[:,cols_1]
        for i in range (len(mylist)):
            X_new[cols[i]] = np.where((X[cols1[i]] > float(mylist[i].split(',')[0])) & (X[cols1[i]] <= float(mylist[i].split(',')[1])), 1, 0)
        return X_new

In [29]:
#Define Modelling pipeline using logisic regression

In [30]:
reg = LogisticRegression(max_iter=1000, class_weight = 'balanced')
woe_transform = WoE_Binning(X)
pipeline = Pipeline(steps=[('woe', woe_transform), ('model', reg)])
pipeline = pipeline.fit(X_train, y_train)
pipeline

Pipeline(steps=[('woe',
                 WoE_Binning(X=       PY_CUSTOMER_ID  CUSTOMER_TYPE  CUSTOMER_AGE  EDUCATION_ID  CONST_CODE  \
0                  51            166           0.0           442         122   
1                  58            168           0.0           442         123   
2                 100            166           0.0           442         122   
3                 107            168          59.0           442         123   
4                 107            168          59.0           442         123   
...               ...            ...           ...           ...         ...   
55842          126060            166          36.0           439         122   
55843          126061            166          39.0           438         122   
55844          126062            166          26.0           438         122   
55845          126064            166          37.0           438         122   
55846          126065            166          45.0           439 

In [31]:
y_pred_log = pipeline.predict(X_test)
accuracy_score(y_test,y_pred_log)

0.9625783348254252

In [32]:
#Testing accuracy score on different algorithm

In [33]:
#1.Naive Bayes Algorithm
from sklearn.naive_bayes import GaussianNB
classifier_nb = GaussianNB()
woe_transform1 = WoE_Binning(X)
classifier_nb = Pipeline(steps=[('woe', woe_transform1), ('model', classifier_nb)])
classifier_nb = classifier_nb.fit(X_train, y_train)
y_pred_NB = classifier_nb.predict(X_test)
accuracy_score(y_test,y_pred_NB)

0.9630259623992838

In [34]:
#2.Decision Tree Algorithm
from sklearn.tree import DecisionTreeClassifier
classifier_dt = DecisionTreeClassifier(criterion='entropy')
woe_transform1 = WoE_Binning(X)
classifier_dt= Pipeline(steps=[('woe', woe_transform1), ('model', classifier_dt)])
classifier_dt = classifier_dt.fit(X_train, y_train)
y_pred_dt = classifier_dt.predict(X_test)
accuracy_score(y_test,y_pred_dt)

0.9878245299910474

In [35]:
#3.Random Forest Algorithm
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(criterion='entropy')
woe_transform1 = WoE_Binning(X)
classifier_rf= Pipeline(steps=[('woe', woe_transform1), ('model', classifier_rf)])
classifier_rf = classifier_rf.fit(X_train, y_train)
y_pred_rf = classifier_rf.predict(X_test)
accuracy_score(y_test,y_pred_rf)

0.990152193375112

In [36]:
# classifier_rf['model'].coef_
# #models like Random Forest do not fit a hyperplane. Hyperplanes are decision boundaries that help classify the data points

In [37]:
# first create a transformed training set through our WoE_Binning custom class
X_train_woe_transformed = woe_transform.fit_transform(X_train)
X_train_woe_transformed

,DEBIT_CARD_FLAG:N,DEBIT_CARD_FLAG:R,DEBIT_CARD_FLAG:Y,CASH_SECURITY_FLAG:N,CASH_SECURITY_FLAG:Y,KYC_COMPLETE_FLAG:N,KYC_COMPLETE_FLAG:Y,LIEN_FLAG:N,LIEN_FLAG:Y,NPA_FLAG:N,NPA_FLAG:Y,PAN:NO,PAN:YES,AC_SECURED_FLAG:N,AC_SECURED_FLAG:Y,AADHAAR_CARD:NO,AADHAAR_CARD:YES,MOBILE_NO:NO,MOBILE_NO:YES,MARTIAL_STATUS:D,MARTIAL_STATUS:M,MARTIAL_STATUS:N,MARTIAL_STATUS:U,MARTIAL_STATUS:W,CUSTOMER_GENDER:F,CUSTOMER_GENDER:M,CUSTOMER_GENDER:N,STATUS_CODE:A,STATUS_CODE:C,STATUS_CODE:D,STATUS_CODE:H,STATUS_CODE:T,"OVERDUE_PRD:-0.161, 6.435","OVERDUE_PRD:6.435, 12.87","OVERDUE_PRD:12.87, 19.304","OVERDUE_PRD:19.304, 25.739","OVERDUE_PRD:25.739, 32.174","OVERDUE_PRD:32.174, 38.609","OVERDUE_PRD:38.609, 45.044","OVERDUE_PRD:45.044, 51.478","OVERDUE_PRD:51.478, 57.913","OVERDUE_PRD:57.913, 64.348","OVERDUE_PRD:64.348, 70.783","OVERDUE_PRD:70.783, 77.218","OVERDUE_PRD:83.652, 90.087","PERIOD_MONTHS:-0.999, 39.96","PERIOD_MONTHS:39.96, 79.92","PERIOD_MONTHS:79.92, 119.88","PERIOD_MONTHS:119.88, 159.84","PERIOD_MONTHS:159.84, 199.8","RISK_TYPE_ID:446.766, 456.36","RISK_TYPE_ID:671.64, 681.0","INTEREST_RATE:7.92, 8.64","INTEREST_RATE:9.36, 10.08","INTEREST_RATE:10.8, 11.52","INTEREST_RATE:11.52, 12.24","INTEREST_RATE:12.96, 13.68","INTEREST_RATE:13.68, 14.4","INTEREST_RATE:14.4, 15.12","INTEREST_RATE:15.84, 16.56","INTEREST_RATE:16.56, 17.28","INTEREST_RATE:17.28, 18.0","PERIOD_YEARS:-0.015, 0.6","PERIOD_YEARS:2.4, 3.0","PERIOD_YEARS:4.8, 5.4","TOTAL_NO_OF_LOAN_ACCOUNT:0.919, 4.24","TOTAL_NO_OF_LOAN_ACCOUNT:4.24, 7.48","TOTAL_NO_OF_LOAN_ACCOUNT:7.48, 10.72","TOTAL_NO_OF_LOAN_ACCOUNT:10.72, 13.96","TOTAL_NO_OF_LOAN_ACCOUNT:20.44, 23.68","AVG_BAL_IN_ACCOUNT:-611058.128, 24442325.129","AVG_BAL_IN_ACCOUNT:24442325.129, 48884650.258","AVG_BAL_IN_ACCOUNT:48884650.258, 73326975.388","AVG_BAL_IN_ACCOUNT:73326975.388, 97769300.517","AVG_BAL_IN_ACCOUNT:97769300.517, 122211625.646","AVG_BAL_IN_ACCOUNT:171096275.904, 195538601.034","PERIOD_DAYS:-0.181, 7.24","TOTAL_NO_OF_ACCOUNT:0.644, 15.24","TOTAL_NO_OF_ACCOUNT:15.24, 29.48","TOTAL_NO_OF_ACCOUNT:29.48, 43.72","TOTAL_NO_OF_ACCOUNT:57.96, 72.2","OVERDUE_AMT:-16415.885, 656635.388","OVERDUE_AMT:656635.388, 1313270.776","OVERDUE_AMT:1313270.776, 1969906.164","OVERDUE_AMT:2626541.552, 3283176.94","IDENTIFICATION_ID:-0.946, 37.84","IDENTIFICATION_ID:416.24, 454.08","IDENTIFICATION_ID:908.16, 946.0","TOTAL_NOOF_DR_TRAN_CNT_SAVING:-1.022, 40.88","TOTAL_NOOF_DR_TRAN_CNT_SAVING:40.88, 81.76","TOTAL_NOOF_DR_TRAN_CNT_SAVING:81.76, 122.64","TOTAL_NOOF_DR_TRAN_CNT_SAVING:122.64, 163.52","TOTAL_NOOF_DR_TRF_TRAN_CNT:-2.806, 112.24"
46132,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
804,1,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1
41788,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
34457,1,0,0,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
32692,1,0,0,1,0,0,1,1,0,1,0,0,1,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41732,1,0,0,1,0,1,0,1,0,0,1,1,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0

In [38]:
# Calculate Intercept & Coefficients From the pipeline model on logistics regression

In [39]:
# Store the column names in X_train as a list
feature_name = X_train_woe_transformed.columns.values
# Create a summary table of our logistic regression model
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
# Create a new column in the dataframe, called 'Coefficients', with row values the transposed coefficients from the 'LogisticRegression' model
summary_table['Coefficients'] = np.transpose(pipeline['model'].coef_)
# Increase the index of every row of the dataframe with 1 to store our model intercept in 1st row
summary_table.index = summary_table.index + 1
# Assign our model intercept to this new row
summary_table.loc[0] = ['Intercept', pipeline['model'].intercept_[0]]
# Sort the dataframe by index
summary_table.sort_index(inplace = True)
summary_table

,Feature name,Coefficients
0,Intercept,8.732330
1,DEBIT_CARD_FLAG:N,-0.291300
2,DEBIT_CARD_FLAG:R,0.735774
3,DEBIT_CARD_FLAG:Y,-0.445600
4,CASH_SECURITY_FLAG:N,0.079187
5,CASH_SECURITY_FLAG:Y,-0.080314
6,KYC_COMPLETE_FLAG:N,0.011158
7,KYC_COMPLETE_FLAG:Y,-0.012285
8,LIEN_FLAG:N,-0.497266
9,LIEN_FLAG:Y,0.496139


In [40]:
#Prediction on Testing data

In [41]:
# make preditions on our test set
y_hat_test = pipeline.predict(X_test)
# get the predicted probabilities
y_hat_test_proba = pipeline.predict_proba(X_test)
# select the probabilities of only the positive class (class 1 - default) 
y_hat_test_proba = y_hat_test_proba[:][: , 1]
# we will now create a new DF with actual classes and the predicted probabilities
# create a temp y_test DF to reset its index to allow proper concaternation with y_hat_test_proba
y_test_temp = y_test.copy()
y_test_temp.reset_index(drop = True, inplace = True)
y_test_proba = pd.concat([y_test_temp, pd.DataFrame(y_hat_test_proba)], axis = 1)
# check the shape to make sure the number of rows is same as that in y_test
y_test_proba.shape
# Rename the columns
y_test_proba.columns = ['y_test_class_actual', 'y_hat_test_proba']
# Makes the index of one dataframe equal to the index of another dataframe.
y_test_proba.index = X_test.index
y_test_proba

,y_test_class_actual,y_hat_test_proba
27194,1,0.999815
15591,1,0.999446
26476,1,0.999363
39516,1,0.999939
25925,1,0.999564
...,...,...
9249,1,0.999993
9771,1,0.999990
45987,1,0.984174
34957,1,0.999234


In [42]:
#Assign a threshold value to differentiate good with bad

In [43]:
tr = 0.5
# crate a new column for the predicted class based on predicted probabilities and threshold
# We will determine this optimat threshold later in this project
y_test_proba['y_test_class_predicted'] = np.where(y_test_proba['y_hat_test_proba'] > tr, 1, 0)
# create the confusion matrix
confusion_matrix(y_test_proba['y_test_class_actual'], y_test_proba['y_test_class_predicted'])
# get the values required to plot a ROC curve
fpr, tpr, thresholds = roc_curve(y_test_proba['y_test_class_actual'], y_test_proba['y_hat_test_proba'])
# Calculate the Area Under the Receiver Operating Characteristic Curve (AUROC) on our test set
AUROC = roc_auc_score(y_test_proba['y_test_class_actual'], y_test_proba['y_hat_test_proba'])
# calculate Gini from AUROC
Gini = AUROC * 2 - 1

In [44]:
# Concatenates two dataframes.
df_scorecard = pd.concat([summary_table])
# We reset the index of a dataframe.
df_scorecard.reset_index(inplace = True)
# create a new column, called 'Original feature name', which contains the value of the 'Feature name' column, up to the column symbol.
df_scorecard['Original feature name'] = df_scorecard['Feature name'].str.split(':').str[0]
df_scorecard

,index,Feature name,Coefficients,Original feature name
0,0,Intercept,8.732330,Intercept
1,1,DEBIT_CARD_FLAG:N,-0.291300,DEBIT_CARD_FLAG
2,2,DEBIT_CARD_FLAG:R,0.735774,DEBIT_CARD_FLAG
3,3,DEBIT_CARD_FLAG:Y,-0.445600,DEBIT_CARD_FLAG
4,4,CASH_SECURITY_FLAG:N,0.079187,CASH_SECURITY_FLAG
5,5,CASH_SECURITY_FLAG:Y,-0.080314,CASH_SECURITY_FLAG
6,6,KYC_COMPLETE_FLAG:N,0.011158,KYC_COMPLETE_FLAG
7,7,KYC_COMPLETE_FLAG:Y,-0.012285,KYC_COMPLETE_FLAG
8,8,LIEN_FLAG:N,-0.497266,LIEN_FLAG
9,9,LIEN_FLAG:Y,0.496139,LIEN_FLAG


In [45]:
# Define the min and max threshholds for our scorecard

In [46]:
min_score = 300
max_score = 850
# calculate the sum of the minimum coefficients of each category within the original feature name
min_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].min().sum()
# calculate the sum of the maximum coefficients of each category within the original feature name
max_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].max().sum()
# create a new columns that has the imputed calculated Score based on the multiplication of the coefficient by the ratio of the differences between
# maximum & minimum score and maximum & minimum sum of cefficients.
df_scorecard['Score - Calculation'] = df_scorecard['Coefficients'] * (max_score - min_score) / (max_sum_coef - min_sum_coef)
# update the calculated score of the Intercept (i.e. the default score for each loan)
df_scorecard.loc[0, 'Score - Calculation'] = ((df_scorecard.loc[0,'Coefficients'] - min_sum_coef) / (max_sum_coef - min_sum_coef)) * (max_score - min_score) + min_score
# round the values of the 'Score - Calculation' column and store them in a new column
df_scorecard['Score - Preliminary'] = df_scorecard['Score - Calculation'].round()
print(df_scorecard['Score - Preliminary'])
# check the min and max possible scores of our scorecard
min_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].min().sum()
max_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].max().sum()

df_scorecard['Difference'] = df_scorecard['Score - Preliminary'] - df_scorecard['Score - Calculation']

#First create a transformed test set through our WoE_Binning custom class and insert an Intercept column
# look like we can get by deducting 1 from the Intercept
df_scorecard['Score - Final'] = df_scorecard['Score - Preliminary']
# Recheck min and max possible scores
print(df_scorecard.groupby('Original feature name')['Score - Final'].min().sum())
print(df_scorecard.groupby('Original feature name')['Score - Final'].max().sum())


0     676.0
1      -4.0
2      10.0
3      -6.0
4       1.0
5      -1.0
6       0.0
7      -0.0
8      -7.0
9       7.0
10     59.0
11    -59.0
12      1.0
13     -1.0
14     -8.0
15      8.0
16      4.0
17     -4.0
18     -3.0
19      3.0
20     -7.0
21      1.0
22     11.0
23     -8.0
24      3.0
25     12.0
26      3.0
27    -16.0
28      1.0
29    -11.0
30     10.0
31      0.0
32      0.0
33     50.0
34     31.0
35     -5.0
36     -5.0
37     -2.0
38      8.0
39      9.0
40    -10.0
41     -8.0
42    -22.0
43    -17.0
44     -8.0
45     -6.0
46     12.0
47     -3.0
48     -2.0
49     -1.0
50     -7.0
51      6.0
52     -6.0
53    -35.0
54    -31.0
55     -3.0
56      4.0
57     14.0
58     -9.0
59     -5.0
60    -27.0
61    -21.0
62    -23.0
63     -1.0
64      6.0
65    -19.0
66    -25.0
67    -17.0
68    -10.0
69    -12.0
70    -30.0
71     13.0
72     -3.0
73      1.0
74    -14.0
75      3.0
76      8.0
77    -15.0
78    -24.0
79      2.0
80     -6.0
81    -23.0
82    -16.0
83  

In [47]:
# first create a transformed test set through our WoE_Binning custom class
X_test_woe_transformed = woe_transform.fit_transform(X_test)
# insert an Intercept column in its beginning to align with the # of rows in scorecard
X_test_woe_transformed.insert(0, 'Intercept', 1)
X_test_woe_transformed.head()
# get the list of our final scorecard scores
scorecard_scores = df_scorecard['Score - Final']
# check the shapes of test set and scorecard before doing matrix dot multiplication
# print(X_test_woe_transformed.shape)
# print(scorecard_scores.shape)
scorecard_scores = scorecard_scores.values.reshape(len(scorecard_scores), 1)
print(X_test_woe_transformed.shape)
print(scorecard_scores.shape)

(11170, 94)
(94, 1)


In [48]:
#Setting loan approval cut-offs
# Calculate Youden's J-Statistic to identify the best threshhold
J = tpr - fpr
# locate the index of the largest J
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold: %f' % (best_thresh))

Best Threshold: 0.387510


In [49]:
# update the threshold value
tr = best_thresh
# crate a new column for the predicted class based on predicted probabilities and threshold
y_test_proba['y_test_class_predicted'] = np.where(y_test_proba['y_hat_test_proba'] > tr,1, 0)
# create the confusion matrix
confusion_matrix(y_test_proba['y_test_class_actual'], y_test_proba['y_test_class_predicted'])
# create a new DF comprising of the thresholds from the ROC output
df_cutoffs = pd.DataFrame(thresholds, columns = ['thresholds'])
# calcue Score corresponding to each threshold
df_cutoffs['Score'] = ((np.log(df_cutoffs['thresholds'] / (1 - df_cutoffs['thresholds'])) - min_sum_coef) * 
                       ((max_score - min_score) / (max_sum_coef - min_sum_coef)) + min_score).round()


C:\Users\vishal.lote\Anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [50]:
#Define approval and rejection rate using best threshhold

In [51]:
def n_approved(p):
    return np.where(y_test_proba['y_hat_test_proba'] >= p, 1, 0).sum()
# Assuming that all credit applications above a given probability of being 'good' will be approved,
# when we apply the 'n_approved' function to a threshold, it will return the number of approved applications.
# Thus, here we calculate the number of approved appliations for all thresholds.
df_cutoffs['N Approved'] = df_cutoffs['thresholds'].apply(n_approved)
# Then, we calculate the number of rejected applications for each threshold.
# It is the difference between the total number of applications and the approved applications for that threshold.
df_cutoffs['N Rejected'] = y_test_proba['y_hat_test_proba'].shape[0] - df_cutoffs['N Approved']
# Approval rate equalts the ratio of the approved applications and all applications.
df_cutoffs['Approval Rate'] = df_cutoffs['N Approved'] / y_test_proba['y_hat_test_proba'].shape[0]
# Rejection rate equals one minus approval rate.
df_cutoffs['Rejection Rate'] = 1 - df_cutoffs['Approval Rate']
# let's have a look at the approval and rejection rates at our ideal threshold
print(df_cutoffs[df_cutoffs['thresholds'].between(best_thresh, best_thresh)])

      thresholds  Score  N Approved  N Rejected  Approval Rate  Rejection Rate
1893     0.38751  555.0       10568         602       0.946106        0.053894


In [52]:
#  WoE_Binning custom class on individual customer data

In [53]:
woe_transform1  = WoE_Binning(X12)
# first create a transformed test set through our WoE_Binning custom class
X_test_woe_transformed1 = woe_transform1.fit_transform(X12)
# insert an Intercept column in its beginning to align with the # of rows in scorecard
X_test_woe_transformed1.insert(0, 'Intercept', 1)
X_test_woe_transformed1.fillna(value = 0, inplace = True)
X_test_woe_transformed1

,Intercept,DEBIT_CARD_FLAG:N,DEBIT_CARD_FLAG:R,DEBIT_CARD_FLAG:Y,CASH_SECURITY_FLAG:N,CASH_SECURITY_FLAG:Y,KYC_COMPLETE_FLAG:N,KYC_COMPLETE_FLAG:Y,LIEN_FLAG:N,LIEN_FLAG:Y,NPA_FLAG:N,NPA_FLAG:Y,PAN:NO,PAN:YES,AC_SECURED_FLAG:N,AC_SECURED_FLAG:Y,AADHAAR_CARD:NO,AADHAAR_CARD:YES,MOBILE_NO:NO,MOBILE_NO:YES,MARTIAL_STATUS:D,MARTIAL_STATUS:M,MARTIAL_STATUS:N,MARTIAL_STATUS:U,MARTIAL_STATUS:W,CUSTOMER_GENDER:F,CUSTOMER_GENDER:M,CUSTOMER_GENDER:N,STATUS_CODE:A,STATUS_CODE:C,STATUS_CODE:D,STATUS_CODE:H,STATUS_CODE:T,"OVERDUE_PRD:-0.161, 6.435","OVERDUE_PRD:6.435, 12.87","OVERDUE_PRD:12.87, 19.304","OVERDUE_PRD:19.304, 25.739","OVERDUE_PRD:25.739, 32.174","OVERDUE_PRD:32.174, 38.609","OVERDUE_PRD:38.609, 45.044","OVERDUE_PRD:45.044, 51.478","OVERDUE_PRD:51.478, 57.913","OVERDUE_PRD:57.913, 64.348","OVERDUE_PRD:64.348, 70.783","OVERDUE_PRD:70.783, 77.218","OVERDUE_PRD:83.652, 90.087","PERIOD_MONTHS:-0.999, 39.96","PERIOD_MONTHS:39.96, 79.92","PERIOD_MONTHS:79.92, 119.88","PERIOD_MONTHS:119.88, 159.84","PERIOD_MONTHS:159.84, 199.8","RISK_TYPE_ID:446.766, 456.36","RISK_TYPE_ID:671.64, 681.0","INTEREST_RATE:7.92, 8.64","INTEREST_RATE:9.36, 10.08","INTEREST_RATE:10.8, 11.52","INTEREST_RATE:11.52, 12.24","INTEREST_RATE:12.96, 13.68","INTEREST_RATE:13.68, 14.4","INTEREST_RATE:14.4, 15.12","INTEREST_RATE:15.84, 16.56","INTEREST_RATE:16.56, 17.28","INTEREST_RATE:17.28, 18.0","PERIOD_YEARS:-0.015, 0.6","PERIOD_YEARS:2.4, 3.0","PERIOD_YEARS:4.8, 5.4","TOTAL_NO_OF_LOAN_ACCOUNT:0.919, 4.24","TOTAL_NO_OF_LOAN_ACCOUNT:4.24, 7.48","TOTAL_NO_OF_LOAN_ACCOUNT:7.48, 10.72","TOTAL_NO_OF_LOAN_ACCOUNT:10.72, 13.96","TOTAL_NO_OF_LOAN_ACCOUNT:20.44, 23.68","AVG_BAL_IN_ACCOUNT:-611058.128, 24442325.129","AVG_BAL_IN_ACCOUNT:24442325.129, 48884650.258","AVG_BAL_IN_ACCOUNT:48884650.258, 73326975.388","AVG_BAL_IN_ACCOUNT:73326975.388, 97769300.517","AVG_BAL_IN_ACCOUNT:97769300.517, 122211625.646","AVG_BAL_IN_ACCOUNT:171096275.904, 195538601.034","PERIOD_DAYS:-0.181, 7.24","TOTAL_NO_OF_ACCOUNT:0.644, 15.24","TOTAL_NO_OF_ACCOUNT:15.24, 29.48","TOTAL_NO_OF_ACCOUNT:29.48, 43.72","TOTAL_NO_OF_ACCOUNT:57.96, 72.2","OVERDUE_AMT:-16415.885, 656635.388","OVERDUE_AMT:656635.388, 1313270.776","OVERDUE_AMT:1313270.776, 1969906.164","OVERDUE_AMT:2626541.552, 3283176.94","IDENTIFICATION_ID:-0.946, 37.84","IDENTIFICATION_ID:416.24, 454.08","IDENTIFICATION_ID:908.16, 946.0","TOTAL_NOOF_DR_TRAN_CNT_SAVING:-1.022, 40.88","TOTAL_NOOF_DR_TRAN_CNT_SAVING:40.88, 81.76","TOTAL_NOOF_DR_TRAN_CNT_SAVING:81.76, 122.64","TOTAL_NOOF_DR_TRAN_CNT_SAVING:122.64, 163.52","TOTAL_NOOF_DR_TRF_TRAN_CNT:-2.806, 112.24"
15,1,1,0,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
16,1,1,0,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
17,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
18,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1


In [54]:
#Predict the Credit Score and NPA status of individual customer

In [55]:
credit_score = X_test_woe_transformed1.dot(scorecard_scores)
print('Customer average Credit score: ' ,round(credit_score.mean().max()))
reg.fit(X_train1, y_train)
array = reg.predict(X1)
for i in array:
    if (i == 1):
        print ("NPA: NO")
    else:
        print ("NPA: YES")
print('Customer Credit score: ' ,round(credit_score))

Customer average Credit score:  570
NPA: YES
NPA: YES
NPA: NO
NPA: NO
Customer Credit score:          0
15  498.0
16  498.0
17  658.0
18  627.0


C:\Users\vishal.lote\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
